# Use topsApp.py to create an interferogram

We've picked two scenes from the same relative orbit over Mt. Rainier for this example ([ASF Search Link](https://search.asf.alaska.edu/#/?zoom=7.927&center=-121.590,46.245&polygon=POLYGON((-121.8771%2046.7881,-121.6408%2046.7881,-121.6408%2046.8958,-121.8771%2046.8958,-121.8771%2046.7881))&productTypes=SLC&start=2021-06-01T07:00:00Z&beamModes=IW&flightDirs=Ascending&resultsLoaded=true&granule=S1A_IW_SLC__1SDV_20210729T015357_20210729T015424_038986_04998B_C323-SLC)):

S1A_IW_SLC__1SDV_20210729T015357_20210729T015424_038986_04998B_C323 
S1B_IW_SLC__1SDV_20210605T015304_20210605T015331_027215_03403B_6C85


## Configure ISCE2 software

https://github.com/isce-framework/isce2

Fortunately we can install everything from conda-forge! We've created a lock file to ensure a reproducible environment is created with locked package versions.

```
conda env create -f environment.yml
conda activate isce2
conda list --explicit > conda-linux-64.lock
```

**Uncomment and run the following cell, then Kernel --> Change Kernel --> isce2. NOTE: you may need log out and log back in for the new kernel to appear.**

In [1]:
import os
if not os.path.exists('/home/studio-lab-user/.conda/envs/isce2'):
    %conda create -y -n isce2 --file conda-linux-64.lock

In [2]:
import isce

# Set Environment variables so that you can call ISCE from the command line
os.environ['ISCE_HOME'] = os.path.dirname(isce.__file__)
os.environ['ISCE_ROOT'] = os.path.dirname(os.environ['ISCE_HOME'])
os.environ['PATH']+='{ISCE_HOME}/bin:{ISCE_HOME}/applications'.format(**os.environ)
print(os.environ['PATH'])

This is the Open Source version of ISCE.
Some of the workflows depend on a separate licensed package.
To obtain the licensed package, please make a request for ISCE
through the website: https://download.jpl.nasa.gov/ops/request/index.cfm.
Alternatively, if you are a member, or can become a member of WinSAR
you may be able to obtain access to a version of the licensed sofware at
https://winsar.unavco.org/software/isce
/home/studio-lab-user/.conda/envs/isce2/bin:/opt/conda/condabin:/opt/conda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin/home/studio-lab-user/.conda/envs/isce2/lib/python3.9/site-packages/isce/bin:/home/studio-lab-user/.conda/envs/isce2/lib/python3.9/site-packages/isce/applications


In [3]:
!which topsApp.py

/home/studio-lab-user/.conda/envs/isce2/lib/python3.9/site-packages/isce/applications/topsApp.py


## Configure processing


The Alaska Satellite Facility stores Sentinel-1 SLC data in AWS us-west-2. Sagemaker StudioLab runs in AWS us-east-2. While it's not optimal to transfer data across datacenters, it should be pretty quick.

In [4]:
!nproc

4


In [5]:
!free -h

              total        used        free      shared  buff/cache   available
Mem:           15Gi       794Mi       7.3Gi       0.0Ki       7.4Gi        14Gi
Swap:            0B          0B          0B


In [6]:
!df -h

Filesystem      Size  Used Avail Use% Mounted on
overlay          27G   12K   27G   1% /
tmpfs            64M     0   64M   0% /dev
tmpfs           7.8G     0  7.8G   0% /sys/fs/cgroup
shm             4.0G     0  4.0G   0% /dev/shm
/dev/nvme0n1p1   50G   14G   37G  27% /opt/.sagemakerinternal
/dev/nvme1n1     25G   12G   14G  47% /home/studio-lab-user
devtmpfs        7.8G     0  7.8G   0% /dev/tty
tmpfs           7.8G     0  7.8G   0% /proc/acpi
tmpfs           7.8G     0  7.8G   0% /sys/firmware


In [7]:
# do our processing in a temporary folder
os.chdir('/tmp')

### Download SLC data


Search for the frames you want to process at https://search.asf.alaska.edu, or use an programmatic API https://github.com/asfadmin/Discovery-asf_search

In [8]:
import getpass

# Update this with your NASA Earthdata login to download SLC data
EARTHDATA_LOGIN = "scottyhq"
EARTHDATA_PASSWORD = getpass.getpass()

 ····················


In [9]:
# Create a persistent .netrc file for downloading NASA datasets (ISCE retrieves SRTM by default)
# https://wiki.earthdata.nasa.gov/display/EL/How+To+Access+Data+With+cURL+And+Wget
!echo "machine urs.earthdata.nasa.gov login {EARTHDATA_LOGIN} password {EARTHDATA_PASSWORD}" > ~/.netrc
!chmod 0600 ~/.netrc

In [10]:
%%time 

# NOTE: can set download to multiple processes for parallelism. 2 SLCs ~9GB of data in ~2-10min

import asf_search

# prevent DEBUG output from download
import logging
logging.getLogger("urllib3").setLevel(logging.WARNING)

reference = 'S1B_IW_SLC__1SDV_20210605T015304_20210605T015331_027215_03403B_6C85'
secondary = 'S1A_IW_SLC__1SDV_20210729T015357_20210729T015424_038986_04998B_C323'
granules = [reference, secondary]

results = asf_search.granule_search(granules)
session = asf_search.ASFSession().auth_with_creds(EARTHDATA_LOGIN, EARTHDATA_PASSWORD)
results.download(path='/tmp', processes=2, session=session)

CPU times: user 298 ms, sys: 202 ms, total: 499 ms
Wall time: 2min 16s


### Download precise orbits 

ISCE has a contributed script to facilitate this


In [11]:
%%bash

wget -nc https://raw.githubusercontent.com/isce-framework/isce2/main/contrib/stack/topsStack/fetchOrbit.py
chmod +x fetchOrbit.py

--2022-01-04 17:21:52--  https://raw.githubusercontent.com/isce-framework/isce2/main/contrib/stack/topsStack/fetchOrbit.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5054 (4.9K) [text/plain]
Saving to: 'fetchOrbit.py'

     0K ....                                                  100% 43.2M=0s

2022-01-04 17:21:52 (43.2 MB/s) - 'fetchOrbit.py' saved [5054/5054]



In [12]:
for file in granules:
    os.system(f'./fetchOrbit.py -i {file}')

Reference time:  2021-06-05 01:53:31
Satellite name:  S1B
Reference time:  2021-07-29 01:54:24
Satellite name:  S1A


### Download digital elevation model

ISCE has a utility script to download an SRTM DEM and convert to Ellipsoid heights

In [13]:
!which dem.py

/home/studio-lab-user/.conda/envs/isce2/lib/python3.9/site-packages/isce/applications/dem.py


In [14]:
!dem.py -c -b 45 48 -123 -120

This is the Open Source version of ISCE.
Some of the workflows depend on a separate licensed package.
To obtain the licensed package, please make a request for ISCE
through the website: https://download.jpl.nasa.gov/ops/request/index.cfm.
Alternatively, if you are a member, or can become a member of WinSAR
you may be able to obtain access to a version of the licensed sofware at
https://winsar.unavco.org/software/isce
curl -n  -L -c $HOME/.earthdatacookie -b $HOME/.earthdatacookie -k -f -O http://e4ftl01.cr.usgs.gov/MEASURES/SRTMGL1.003/2000.02.11/N47W123.SRTMGL1.hgt.zip
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   267  100   267    0     0   3359      0 --:--:-- --:--:-- --:--:--  3379
100 7711k  100 7711k    0     0   712k      0  0:00:10  0:00:10 --:--:-- 1697k
curl -n  -L -c $HOME/.earthdatacookie -b $HOME/.earthdatacookie -k -f -O http://e4ftl01.cr.usgs.gov/MEASURE

### Write topsApp input file

In [15]:
with open('topsApp.xml', 'w') as f:
    f.write(f"""<?xml version="1.0" encoding="UTF-8"?>
<topsApp>
  <component name="topsinsar">
    <property name="Sensor name">SENTINEL1</property>
    <component name="reference">
        <property name="orbit directory">./</property>
        <property name="output directory">reference</property>
        <property name="safe">{reference}.zip</property>
    </component>
    <component name="secondary">
        <property name="orbit directory">./</property>
        <property name="output directory">secondary</property>
        <property name="safe">{secondary}.zip</property>
    </component>
    <property name="demfilename">demLat_N45_N48_Lon_W123_W120.dem.wgs84</property>
    <property name="useGPU">False</property>
    <property name="swaths">[1]</property>
    <!-- <property name="range looks">7</property> -->
    <!-- <property name="azimuth looks">3</property> -->
    <!-- <property name="region of interest">[South, North, West, East]</property> -->
    <property name="do esd">False</property>
    <property name="do ionospherecorrection">False</property>
    <property name="do unwrap">True</property>
    <property name="unwrapper name">snaphu_mcf</property>
    <property name="do denseoffsets">False</property>
    <!--<property name="geocode demfilename">path_to_your_dem</property> -->
    <!--property name="geocode list">['merged/phsig.cor', 'merged/filt_topophase.unw', 'merged/los.rdr', 'merged/topophase.flat', 'merged/filt_topophase.flat','merged/topophase.cor','merged/filt_topophase.unw.conncomp']</property> -->
  </component>
</topsApp>""")

## Process Interferogram

You can either run the entire processing workflow start to finish (`topsApp.py`), or run it step, by step with the following steps in order:

['startup', 'preprocess', 'computeBaselines', 'verifyDEM', 'topo']

['subsetoverlaps', 'coarseoffsets', 'coarseresamp', 'overlapifg', 'prepesd']

['esd', 'rangecoreg', 'fineoffsets', 'fineresamp', 'ion']

['burstifg', 'mergebursts', 'filter', 'unwrap', 'unwrap2stage']

['geocode', 'denseoffsets', 'filteroffsets', 'geocodeoffsets']

**TIP**: right click on a cell and select 'Enable scrolling for Outputs' b/c the topsApp logs have a lot of text. Or use %%capture to store the logs as Python variables. 

In [16]:
# Lots of properties you can set via the XML file
# !topsApp.py --help

In [17]:
# Run steps in sequence
#!topsApp.py --help --steps

In [18]:
%%time

!topsApp.py --end=computeBaselines

This is the Open Source version of ISCE.
Some of the workflows depend on a separate licensed package.
To obtain the licensed package, please make a request for ISCE
through the website: https://download.jpl.nasa.gov/ops/request/index.cfm.
Alternatively, if you are a member, or can become a member of WinSAR
you may be able to obtain access to a version of the licensed sofware at
https://winsar.unavco.org/software/isce
2022-01-04 19:10:25,775 - isce.insar - INFO - ISCE VERSION = 2.5.3, RELEASE_SVN_REVISION = ,RELEASE_DATE = 20210823, CURRENT_SVN_REVISION = 
ISCE VERSION = 2.5.3, RELEASE_SVN_REVISION = ,RELEASE_DATE = 20210823, CURRENT_SVN_REVISION = 
Step processing
Running step startup
None
The currently supported sensors are:  ['SENTINEL1']
Dumping the application's pickle object _insar to file  PICKLE/startup
The remaining steps are (in order):  ['preprocess', 'computeBaselines', 'verifyDEM', 'topo', 'subsetoverlaps', 'coarseoffsets', 'coarseresamp', 'overlapifg', 'prepesd', 'esd', 'r

In [19]:
ls

PICKLE/
S1A_IW_SLC__1SDV_20210729T015357_20210729T015424_038986_04998B_C323.iso.xml
S1A_IW_SLC__1SDV_20210729T015357_20210729T015424_038986_04998B_C323.zip
S1A_OPER_AUX_POEORB_OPOD_20210818T122052_V20210728T225942_20210730T005942.EOF
S1B_IW_SLC__1SDV_20210605T015304_20210605T015331_027215_03403B_6C85.iso.xml
S1B_IW_SLC__1SDV_20210605T015304_20210605T015331_027215_03403B_6C85.zip
S1B_OPER_AUX_POEORB_OPOD_20210625T111713_V20210604T225942_20210606T005942.EOF
demLat_N45_N48_Lon_W123_W120.dem
demLat_N45_N48_Lon_W123_W120.dem.vrt
demLat_N45_N48_Lon_W123_W120.dem.wgs84
demLat_N45_N48_Lon_W123_W120.dem.wgs84.vrt
demLat_N45_N48_Lon_W123_W120.dem.wgs84.xml
demLat_N45_N48_Lon_W123_W120.dem.xml
fetchOrbit.py*
insar.log
isce.log
reference/
secondary/
topsApp.xml
topsProc.xml
topsinsar.log


In [20]:
# This will download 30m SRTM DEM for processing from http://e4ftl01.cr.usgs.gov/MEASURES/SRTMGL1.003
!topsApp.py --dostep=verifyDEM

This is the Open Source version of ISCE.
Some of the workflows depend on a separate licensed package.
To obtain the licensed package, please make a request for ISCE
through the website: https://download.jpl.nasa.gov/ops/request/index.cfm.
Alternatively, if you are a member, or can become a member of WinSAR
you may be able to obtain access to a version of the licensed sofware at
https://winsar.unavco.org/software/isce
2022-01-04 19:10:45,833 - isce.insar - INFO - ISCE VERSION = 2.5.3, RELEASE_SVN_REVISION = ,RELEASE_DATE = 20210823, CURRENT_SVN_REVISION = 
ISCE VERSION = 2.5.3, RELEASE_SVN_REVISION = ,RELEASE_DATE = 20210823, CURRENT_SVN_REVISION = 
Step processing
Running step verifyDEM
Dumping the application's pickle object _insar to file  PICKLE/verifyDEM
The remaining steps are (in order):  ['topo', 'subsetoverlaps', 'coarseoffsets', 'coarseresamp', 'overlapifg', 'prepesd', 'esd', 'rangecoreg', 'fineoffsets', 'fineresamp', 'ion', 'burstifg', 'mergebursts', 'filter', 'unwrap', 'unwr

In [21]:
%%time

# Wall time: ~16min 50s

!topsApp.py --dostep=topo

This is the Open Source version of ISCE.
Some of the workflows depend on a separate licensed package.
To obtain the licensed package, please make a request for ISCE
through the website: https://download.jpl.nasa.gov/ops/request/index.cfm.
Alternatively, if you are a member, or can become a member of WinSAR
you may be able to obtain access to a version of the licensed sofware at
https://winsar.unavco.org/software/isce
2022-01-04 19:10:46,446 - isce.insar - INFO - ISCE VERSION = 2.5.3, RELEASE_SVN_REVISION = ,RELEASE_DATE = 20210823, CURRENT_SVN_REVISION = 
ISCE VERSION = 2.5.3, RELEASE_SVN_REVISION = ,RELEASE_DATE = 20210823, CURRENT_SVN_REVISION = 
Step processing
Running step topo
2022-01-04 19:10:46,810 - isce.zerodop.topozero - WARNING - Default Peg heading set to: -0.2500339047555438
Polynomial Order: 0 - by - 0 
0	
Polynomial Order: 0 - by - 1 
799779	2.32956	
GDAL open (R): /tmp/demLat_N45_N48_Lon_W123_W120.dem.wgs84.vrt
API open (WR): geom_reference/IW1/lat_04.rdr
API open (WR):

In [22]:
%%time

# Wall time: ~10min 20s
!topsApp.py --start=subsetoverlaps --end=mergebursts

This is the Open Source version of ISCE.
Some of the workflows depend on a separate licensed package.
To obtain the licensed package, please make a request for ISCE
through the website: https://download.jpl.nasa.gov/ops/request/index.cfm.
Alternatively, if you are a member, or can become a member of WinSAR
you may be able to obtain access to a version of the licensed sofware at
https://winsar.unavco.org/software/isce
2022-01-04 19:27:31,659 - isce.insar - INFO - ISCE VERSION = 2.5.3, RELEASE_SVN_REVISION = ,RELEASE_DATE = 20210823, CURRENT_SVN_REVISION = 
ISCE VERSION = 2.5.3, RELEASE_SVN_REVISION = ,RELEASE_DATE = 20210823, CURRENT_SVN_REVISION = 
Step processing
Running step subsetoverlaps
Dumping the application's pickle object _insar to file  PICKLE/subsetoverlaps
The remaining steps are (in order):  ['coarseoffsets', 'coarseresamp', 'overlapifg', 'prepesd', 'esd', 'rangecoreg', 'fineoffsets', 'fineresamp', 'ion', 'burstifg', 'mergebursts', 'filter', 'unwrap', 'unwrap2stage', 'geoc

In [25]:
%%time

# Wall time: 7min 56s
!topsApp.py --start=filter --end=geocode

This is the Open Source version of ISCE.
Some of the workflows depend on a separate licensed package.
To obtain the licensed package, please make a request for ISCE
through the website: https://download.jpl.nasa.gov/ops/request/index.cfm.
Alternatively, if you are a member, or can become a member of WinSAR
you may be able to obtain access to a version of the licensed sofware at
https://winsar.unavco.org/software/isce
2022-01-04 20:22:07,088 - isce.insar - INFO - ISCE VERSION = 2.5.3, RELEASE_SVN_REVISION = ,RELEASE_DATE = 20210823, CURRENT_SVN_REVISION = 
ISCE VERSION = 2.5.3, RELEASE_SVN_REVISION = ,RELEASE_DATE = 20210823, CURRENT_SVN_REVISION = 
Step processing
Running step filter
2022-01-04 20:22:07,140 - isce.topsinsar.runFilter - INFO - Applying power-spectral filter
GDAL open (R): merged/topophase.flat.vrt
API open (WR): merged/filt_topophase.flat
2022-01-04 20:22:07,198 - isce.mroipac.filter - DEBUG - width: 1167
2022-01-04 20:22:07,198 - isce.mroipac.filter - DEBUG - length: 1

In [28]:
# Merged interferograms
ls merged/

dem.crop                         los.rdr.xml
dem.crop.vrt                     phsig.cor
dem.crop.xml                     phsig.cor.geo
filt_topophase.flat              phsig.cor.geo.vrt
filt_topophase.flat.geo          phsig.cor.geo.xml
filt_topophase.flat.geo.vrt      phsig.cor.vrt
filt_topophase.flat.geo.xml      phsig.cor.xml
filt_topophase.flat.vrt          reference.slc.full.vrt
filt_topophase.flat.xml          reference.slc.full.xml
filt_topophase.unw               secondary.slc.full.vrt
filt_topophase.unw.conncomp      secondary.slc.full.xml
filt_topophase.unw.conncomp.vrt  topophase.cor
filt_topophase.unw.conncomp.xml  topophase.cor.full.vrt
filt_topophase.unw.geo           topophase.cor.full.xml
filt_topophase.unw.geo.vrt       topophase.cor.geo
filt_topophase.unw.geo.xml       topophase.cor.geo.vrt
filt_topophase.unw.vrt           topophase.cor.geo.xml
filt_topophase.unw.xml           topophase.cor.vrt
lat.rdr.full.vrt                 topophase.cor.xml
lat.rdr.full.xml       

## Visualize results